In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import random
from collections import defaultdict

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Define parameters for federated learning
NUM_FOG_NODES = 5  # Number of fog nodes
NUM_GLOBAL_ROUNDS = 10  # Number of global federated learning rounds
LEARNING_RATE = 0.01  # Learning rate
DATASET_SIZE_PER_NODE = 1000  # Size of each fog node's dataset
PRIORITY_WEIGHTS = {"Q1": 0.5, "Q2": 0.5}  # Weights for priority calculation

# Simulate charging requests and their attributes
def generate_charging_requests(num_requests):
    """Simulate charging requests with SoC and urgency values."""
    requests = []
    for i in range(num_requests):
        request = {
            "id": f"Req-{i+1}",
            "SoC": random.uniform(0, 1),  # State of charge (0 to 100%)
            "Urgency": random.uniform(0, 1),  # Urgency level (0 to 1)
        }
        requests.append(request)
    return requests

# Calculate priorities
def calculate_priorities(requests, weights):
    """Calculate priority scores for charging requests."""
    for req in requests:
        req["Priority"] = weights["Q1"] * (1 - req["SoC"]) + weights["Q2"] * req["Urgency"]
    # Sort requests by priority (descending)
    return sorted(requests, key=lambda x: x["Priority"], reverse=True)

# Simulate local model training
def local_training(data_size, learning_rate):
    """Simulate one round of local training."""
   
    model_update = np.random.randn() * learning_rate / np.sqrt(data_size)
    return model_update
# np.random.randn() : un facteur aléatoire dans la mise à jour du modèle ,
#  learning_rate réduit l'impact de la valeur aléatoire pour éviter des mises à jour trop importantes.

# Aggregate models at the cloud level
def global_aggregation(local_updates, data_sizes):
    """Aggregate local models to form a global model."""
    total_data_size = sum(data_sizes)
    global_model = sum((update * size / total_data_size for update, size in zip(local_updates, data_sizes)))
    return global_model

# Simulate the federated learning process
def federated_learning(num_fog_nodes, num_rounds, learning_rate, dataset_size):
    """Perform federated learning."""
    # Initialize global model
    global_model = np.random.randn()
    print(f"Initial global model: {global_model:.4f}")
    
    # Simulate datasets for fog nodes
    datasets = [dataset_size for _ in range(num_fog_nodes)]
    
    for round_num in range(num_rounds):
        local_updates = []
        # Each fog node trains its model
        #entraîne un modèle local avec les données et le taux d'apprentissage donnés.
        #Il stocke la mise à jour du modèle dans une liste pour un usage ultérieur
        for node_id in range(num_fog_nodes):
            update = local_training(datasets[node_id], learning_rate)
            local_updates.append(update)
        
        # Aggregate updates to update the global model
        global_model += global_aggregation(local_updates, datasets)
        print(f"Round {round_num+1}: Updated global model: {global_model:.4f}")
    
    return global_model

# Simulate VM allocation
def allocate_requests_to_vms(requests, num_vms, vm_capacity):
    """Distribute requests to VMs based on priority and load balancing."""
    vms = {f"VM-{i+1}": {"Load": 0, "Capacity": vm_capacity, "Requests": []} for i in range(num_vms)}
    
    for req in requests:
        # Find VM with the minimum load that can accommodate the request
        #Trouve les VM disponibles qui peuvent gérer chaque demande en vérifiant leur capacité et leur charge actuelle
        suitable_vms = [(vm, info) for vm, info in vms.items() if info["Load"] + 1 <= info["Capacity"]]
        if suitable_vms:
            selected_vm = min(suitable_vms, key=lambda x: x[1]["Load"])[0]
            vms[selected_vm]["Load"] += 1
            vms[selected_vm]["Requests"].append(req["id"])
        else:
            print(f"Request {req['id']} could not be allocated due to capacity constraints.")
    
    return vms

# Main execution
if __name__ == "__main__":
    # Generate charging requests
    num_requests = 15  # Number of charging requests
    charging_requests = generate_charging_requests(num_requests)
    
    # Calculate priorities
    prioritized_requests = calculate_priorities(charging_requests, PRIORITY_WEIGHTS)
    print("Prioritized Charging Requests:")
    for req in prioritized_requests:
        print(f"ID: {req['id']}, Priority: {req['Priority']:.4f}")
    
    # Perform federated learning
    final_global_model = federated_learning(NUM_FOG_NODES, NUM_GLOBAL_ROUNDS, LEARNING_RATE, DATASET_SIZE_PER_NODE)
    print(f"Final global model after {NUM_GLOBAL_ROUNDS} rounds: {final_global_model:.4f}")
    
    # Allocate requests to VMs
    num_vms = 3  # Number of VMs
    vm_capacity = 5  # Capacity of each VM
    vm_allocations = allocate_requests_to_vms(prioritized_requests, num_vms, vm_capacity)
    
    print("\nVM Allocations:")
    for vm, info in vm_allocations.items():
        print(f"{vm}: Load = {info['Load']}, Requests = {info['Requests']}")


Prioritized Charging Requests:
ID: Req-9, Priority: 0.6844
ID: Req-6, Priority: 0.6434
ID: Req-7, Priority: 0.5862
ID: Req-14, Priority: 0.5020
ID: Req-2, Priority: 0.4741
ID: Req-3, Priority: 0.4701
ID: Req-8, Priority: 0.4475
ID: Req-11, Priority: 0.4462
ID: Req-12, Priority: 0.4076
ID: Req-15, Priority: 0.3781
ID: Req-5, Priority: 0.3039
ID: Req-1, Priority: 0.1928
ID: Req-13, Priority: 0.1897
ID: Req-10, Priority: 0.0985
ID: Req-4, Priority: 0.0974
Initial global model: 0.4967
Round 1: Updated global model: 0.4968
Round 2: Updated global model: 0.4969
Round 3: Updated global model: 0.4967
Round 4: Updated global model: 0.4966
Round 5: Updated global model: 0.4964
Round 6: Updated global model: 0.4963
Round 7: Updated global model: 0.4963
Round 8: Updated global model: 0.4962
Round 9: Updated global model: 0.4960
Round 10: Updated global model: 0.4960
Final global model after 10 rounds: 0.4960

VM Allocations:
VM-1: Load = 5, Requests = ['Req-9', 'Req-14', 'Req-8', 'Req-15', 'Req-13